In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler

In [5]:
df = pd.read_csv('../data/merged_orders_products.csv')
feedback = pd.read_csv('../data/blinkit_customer_feedback.csv')

print('Merged orders + products data')
df.head()

Merged orders + products data


,order_id,product_id,quantity,unit_price,customer_id,order_date,promised_delivery_time,actual_delivery_time,delivery_status,order_total,...,store_id,product_name,category,brand,price,mrp,margin_percentage,shelf_life_days,min_stock_level,max_stock_level
0,1961864118,642612,3,517.03,30065862,2024-07-17 08:34:01,2024-07-17 08:52:01,2024-07-17 08:47:01,On Time,3197.07,...,4771,Pet Treats,Pet Care,Pillay-Ahuja,517.03,795.43,35.0,365,16,57
1,1549769649,378676,1,881.42,9573071,2024-05-28 13:14:29,2024-05-28 13:25:29,2024-05-28 13:27:29,On Time,976.55,...,7534,Orange Juice,Cold Drinks & Juices,Baral-Kamdar,881.42,1259.17,30.0,180,19,65
2,9185164487,741341,2,923.84,45477575,2024-09-23 13:07:12,2024-09-23 13:25:12,2024-09-23 13:29:12,On Time,839.05,...,9886,Eggs,Dairy & Breakfast,Prasad LLC,923.84,1154.80,20.0,7,13,62
3,9644738826,561860,1,874.78,88067569,2023-11-24 16:16:56,2023-11-24 16:34:56,2023-11-24 16:33:56,On Time,440.23,...,7917,Orange Juice,Cold Drinks & Juices,Gupta Ltd,874.78,1249.69,30.0,180,21,90
4,5427684290,602241,2,976.55,83298567,2023-11-20 05:00:39,2023-11-20 05:17:39,2023-11-20 05:18:39,On Time,2526.68,...,2741,Nuts,Snacks & Munchies,Bahl-Pau,976.55,1502.38,35.0,90,12,75


In [7]:
feedback.head()

,feedback_id,order_id,customer_id,rating,feedback_text,feedback_category,sentiment,feedback_date
0,2234710,1961864118,30065862,4,"It was okay, nothing special.",Delivery,Neutral,2024-07-17
1,5450964,1549769649,9573071,3,The order was incorrect.,App Experience,Negative,2024-05-28
2,482108,9185164487,45477575,3,"It was okay, nothing special.",App Experience,Neutral,2024-09-23
3,4823104,9644738826,88067569,4,The product met my expectations.,App Experience,Neutral,2023-11-24
4,3537464,5427684290,83298567,3,Product was damaged during delivery.,Delivery,Negative,2023-11-20


Rating-Based Recommendation

In [11]:
#merge feedback with product level data
df_feedback = feedback.merge(df,on=['order_id','customer_id'],how='inner')

In [13]:
#droping missing product names
df_feedback = df_feedback.dropna(subset=["product_name"])

In [15]:
#Average rating per product
product_ratings = df_feedback.groupby("product_name")["rating"].mean().sort_values(ascending=False)

In [17]:
#Count of ratings per product (to filter out low-volume ratings)
rating_counts = df_feedback.groupby("product_name")["rating"].count()

In [19]:
def recommend_top_rated_products(min_reviews=10, top_n=5):
    popular_products = rating_counts[rating_counts >= min_reviews].index
    filtered_ratings = product_ratings.loc[popular_products]
    return filtered_ratings.head(top_n)

In [21]:
recommend_top_rated_products()

product_name
Baby Food     3.321739
Baby Wipes    3.267045
Bananas       3.111111
Biscuits      3.463415
Bread         3.222222
Name: rating, dtype: float64

Personalized: Rated Products Not Yet Bought

In [23]:
def recommend_rated_not_bought(customer_id, min_reviews=10, top_n=5):
    bought = set(df[df["customer_id"] == customer_id]["product_name"])
    popular_products = rating_counts[rating_counts >= min_reviews].index
    top_products = product_ratings.loc[popular_products]
    
    not_bought = top_products[~top_products.index.isin(bought)]
    return not_bought.head(top_n)

In [25]:
#Try for sample customer
sample_customer = df["customer_id"].unique()[0]
print(f"🛍️ Rated Recommendations for Customer {sample_customer}:")
print(recommend_rated_not_bought(sample_customer))

🛍️ Rated Recommendations for Customer 30065862:
product_name
Baby Food     3.321739
Baby Wipes    3.267045
Bananas       3.111111
Biscuits      3.463415
Bread         3.222222
Name: rating, dtype: float64


Recommend New Arrivals to Active Users

In [35]:
# Convert date column
df['order_date'] = pd.to_datetime(df['order_date'])

# Most recent date in dataset
recent_date = df['order_date'].max()

# Build activity metrics
user_activity = df.groupby('customer_id').agg({
    'order_id': 'nunique',
    'order_date': lambda x: (recent_date - x.max()).days
}).rename(columns={'order_id': 'num_orders', 'order_date': 'days_since_last_order'})

#  Recency score = lower days → higher score
user_activity['recency_score'] = 1 / (1 + user_activity['days_since_last_order'])

# Combine score: order frequency + recency
user_activity['activity_score'] = user_activity['num_orders'] + user_activity['recency_score']

# Preview
user_activity.sort_values('activity_score', ascending=False).head()


,num_orders,days_since_last_order,recency_score,activity_score
customer_id,,,,
77869660,9,57,0.017241,9.017241
8791577,8,21,0.045455,8.045455
17805991,8,46,0.021277,8.021277
10562528,7,19,0.050000,7.050000
75213636,7,24,0.040000,7.040000


In [37]:
product_launch_dates = df.groupby('product_name')['order_date'].min().sort_values(ascending=False)

In [41]:
top_5_percent = int(0.05*len(product_launch_dates))
new_arrivals = product_launch_dates.head(top_5_percent).index.tolist()

In [43]:
new_arrivals[:5]

['Iced Tea', 'Cereal']

In [45]:
def recommend_new_arrivals_to_active_users(top_n_users=10, top_n_products=5):
    top_users = user_activity.sort_values('activity_score', ascending=False).head(top_n_users).index
    recommendations = {}
    
    for uid in top_users:
        bought = set(df[df['customer_id'] == uid]['product_name'])
        not_bought = [prod for prod in new_arrivals if prod not in bought]
        recommendations[uid] = not_bought[:top_n_products]
    
    return recommendations

In [47]:
new_arrival_recs = recommend_new_arrivals_to_active_users()

In [51]:
print("🆕 New Arrival Recommendations for Active Users:\n")
for user, products in new_arrival_recs.items():
    print(f"Customer {user}: {products}")

🆕 New Arrival Recommendations for Active Users:

Customer 77869660: ['Cereal']
Customer 8791577: ['Iced Tea', 'Cereal']
Customer 17805991: ['Iced Tea', 'Cereal']
Customer 10562528: ['Iced Tea', 'Cereal']
Customer 75213636: ['Iced Tea', 'Cereal']
Customer 12832151: ['Iced Tea', 'Cereal']
Customer 26285589: ['Cereal']
Customer 21701991: ['Iced Tea', 'Cereal']
Customer 93018527: ['Iced Tea', 'Cereal']
Customer 13604883: ['Iced Tea', 'Cereal']
